In [ ]:
"""MgH+ Polaritonic Potential Energy Surfaces"""

__authors__ = ["Ruby Manderna, Lane Tolley, Figen Suchanek, Jonathan J. Foley, "]
__email__   = ["rmandern@uncc.edu, ptolley1@uncc.edu, suchanekf@wpunj.edu, jfoley19@uncc.edu, "]
__credits__ = ["Ruby Manderna, Lane Tolley, Figen Suchanek, Jonathan J. Foley"]
__copyright__ = "(c) 2008-2020, The Psi4Education Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-02-11"


### Import libraries to be used throughout
# basic psi4 library
import psi4
# pyplot
from matplotlib import pyplot as plt
# numpy
import numpy as np
# scipy
from scipy.interpolate import InterpolatedUnivariateSpline
# linear algebra package from numpy
from numpy import linalg as LA
# time-dependent scf library from psi4 for computing excited states and transition dipole moments
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
#from matplotlib import pyplot as plt
psi4.set_output_file('output.dat', False)


In [ ]:
def compute_energy(mol_tmp, options, theory_string, r0, root):
    mol_str = mol_tmp.replace("**R**", str(r0))
    mol = psi4.geometry(mol_str)
    psi4.set_options(options)
    e, wfn = psi4.energy(theory_string, return_wfn=True, molecule=mol)
    
    if root==0:
        return e
    
    else:
        n_states = root + 1
        # calculate the excited-state energies and save them to a dictionary called 'res'
        res = tdscf_excitations(wfn, states=n_states, triplets = "NONE")
        
        # parse the excitation energies from the 'res' dictionary 
        delta_e = [r["EXCITATION ENERGY"] for r in res]
        
        return e + delta_e[root-1]
    
    
    


def compute_gradient(mol_tmp, options, theory_string, r0, dr, root):
    rv = r0
    r_array = np.array([
        rv - 2 * dr,
        rv - dr,
        rv, 
        rv + dr,
        rv + 2 * dr
    ])
    
    f = np.zeros(5)
    # get h in atomic units
    h = dr / 0.52917721067121
    
    print(r_array)
    for i in range(len(r_array)):
        f[i] = compute_energy(mol_tmp, options, theory_string, r_array[i], rt)
        print(F"Energy is {f[i]} for bondlength {r_array[i]}")
    
    
    energy = f[2]
    
    gradient = (1 * f[0] - 8 * f[1] + 0 * f[2] + 8 * f[3] - 1 * f[4]) / (
            12 * 1.0 * h
    )
    hessian = (-1 * f[0] + 16 * f[1] - 30 * f[2] + 16 * f[3] - 1 * f[4]) / (
            12 * 1.0 * h**2
    )
    
    return energy, gradient, hessian


def optimize_geometry_full_nr(mol_tmp, options, theory_string, r0, dr, root, gradient_tol, step_tol):
    print(
        f" Going to perform a full Newton-Raphson geometry optimization using {theory_string}"
    )
    energy0, gradient0, hessian0 = compute_gradient(mol_tmp, options, theory_string, r0, dr, root)
    print(f" Initial Energy is     {energy0}")
    print(f" Initial bondlength is {r0}")
    print(f" Initial Gradient is   {gradient0}")
    print(f" Initial Hessian is    {hessian0}")
    _delta_x = -gradient0 / hessian0
    print(f" Initial Update is     {_delta_x} ")

    iter_count = 1
    gradient = gradient0
    while np.abs(gradient) > gradient_tol and iter_count < 200:
        if np.abs(_delta_x) < step_tol:
            r0 += 0.5 * _delta_x
        else:
            r0 += _delta_x / np.abs(_delta_x) * step_tol
        energy, gradient, hessian = compute_gradient(mol_tmp, options, theory_string, r0, dr, root)
        print(f" Geometry Update {iter_count}")
        print(f" Bondlength:     {r0}")
        print(f" Energy:         {energy}")
        print(f" Gradient:       {gradient}")
        print(f" Hessian:        {hessian}")
        _delta_x = -gradient / hessian
        print(f" Update:         {_delta_x}")
        iter_count += 1
    
    
    return r0, energy, gradient, hessian


def potential_scan(min_r, max_r, num_r, mol_tmp, options, theory_string, root):
    r_vals = np.linspace(min_r, max_r, num_r)
    
    E_array = []
    for r in r_vals:
        
        Ev = compute_energy(mol_tmp, options, theory_string, r, root)
        E_array.append(Ev)
        
    return r_vals, E_array
        
    

In [ ]:
mol_tmpl = """
Li
H 1 **R**
symmetry c2v
"""

options_dict = {
    "basis": "6-311G**",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
    'save_jk': True,
}

t_string = "b3lyp/6-311G**"

rval = 1.5
rt = 0

E = compute_energy(mol_tmpl, options_dict, t_string, rval, rt)
print(E)

In [ ]:
r_arr, E0_arr = potential_scan(1.0, 3.0, 50, mol_tmpl, options_dict, t_string, 0)
r_arr, E1_arr = potential_scan(1.0, 3.0, 50, mol_tmpl, options_dict, t_string, 1)
#r_arr, E2_arr = potential_scan(1.0, 3.0, 50, mol_tmpl, options_dict, t_string, 2)
plt.plot(r_arr, E0_arr)
plt.plot(r_arr, E1_arr)
plt.plot(r_arr, E2_arr)
plt.show()

In [ ]:
plt.plot(r_arr, np.array(E0_arr)+0.12086)
plt.plot(r_arr, E1_arr)
plt.show()

In [ ]:
rt = 1
energy, gradient, hessian = compute_gradient(mol_tmpl, options_dict, t_string, rval, 0.001, rt)

rf, energyf, gradientf, hessian_f = optimize_geometry_full_nr(mol_tmpl, options_dict, t_string, rval, 0.001, rt, 1e-5, 0.5)

In [ ]:
# set basis
psi4.set_options({
    'basis':'cc-pVDZ'
})

# set the number of electronic states... this is the ground state + n_states more
# we will get 1 excited-state
n_states = 1

# set the number of bond lengths to compute the stretch along
n_geoms = 25

# initialize geometry list
geoms = []

# initialize energy list... note
# there will be the ground state energy + n_states excited state energies
Es = np.zeros((n_states+1, n_geoms))



# generate bond lengths
rs = []
for i in range(0,n_geoms):
    rs.append(1.1 + i*0.1)

# loop over bond lengths
ctr = 0
for i in rs:
    # generate the MgH+ molecule using a z-matrix and set the Mg-H+ bond length
    mol = psi4.geometry("""
    Mg
    H 1 """ + str(i) + """
    symmetry c1
    1 1
    """)
    # save the geometry
    geoms.append(mol.geometry().to_array())
    psi4.set_options({
    'save_jk': True,
    })  
   
    # calculate and save the ground-state energy and wavefunction
    e, wfn = psi4.energy("b3lyp/cc-pVDZ", return_wfn=True, molecule=mol)
    
    # calculate the excited-state energies and save them to a dictionary called 'res'
    res = tdscf_excitations(wfn, states=n_states, triplets = "NONE")
    
    # parse the excitation energies from the 'res' dictionary 
    delta_e = [r["EXCITATION ENERGY"] for r in res]
    
    # parse the transition dipole moment from the 'res' dictionary
    mu = [r["ELECTRIC DIPOLE TRANSITION MOMENT (LEN)"] for r in res]
    Es[0,ctr] = e
    
    # store the results to the respective arrays
    for j in range(0, n_states):
        Es[j+1,ctr] = e + delta_e[j]
        # we only want the z-component which is index 2
        mu_z[j,ctr] = np.absolute(mu[j][2])

    
    # increment the counter!
    ctr += 1
    



In [ ]:
# Take slices of information for Eg, Ee, and mu_eg only
Eg_array = np.copy(Es[0,:])
Ee_array = np.copy(Es[1,:])
tdm_array = np.copy(mu_z[0,:])


assert np.allclose(Eg_array, _expected_Eg_array)
assert np.allclose(Ee_array, _expected_Ee_array)
assert np.allclose(tdm_array, _expected_tdm_array)

In [ ]:
# This will print all of the keys associated with the `res` dictionary.
print(res[0].keys())

Let's plot the ground- and excited-state potential energy surfaces for MgH+ just computed above.

In [ ]:

plt.plot(rs,Eg_array, label='Ground state')
plt.plot(rs,Ee_array, label='Excited state')
plt.legend()
plt.xlabel('Bondlength (Angstroms)')
plt.ylabel('Energy (Hartrees)')
plt.title('DFT/TDDFT Potential Energy Surfaces')
plt.grid(True)
plt.show()

#### Building the Rabi model Hamiltonian 

1. Fit each surface to a spline

2. Define functions that build the Rabi Hamiltonian and extract the eigenstates from them.  For reasons of convenience with the plotting widget, we will create two almost identical functions that extract the lower-polariton and upper-polariton roots separately. 

3. (Separate notebook) Define a widget that allows the display of polaritonic potential energy surfaces with variable values of the coupling strength $g$.  

##### Step 1: Fitting a spline
We will fit a cubic spline to the data obtained for the ground- and excited-state potential energy surfaces ($E_g(R)$ and $E_e(R)$) obtained from the TDDFT calculations.  A cubic spline is a simple model function of the form $f(R) = a + bR + cR^2 + dR^3$ that allows estimation of the potential energy curves at arbitrary values of $R$ within the finite range scanned by the *ab initio* calculations.  We will do the same for the transition dipole moment, $\mu_{ge}(R)$.

The syntax for generating a cubic spline is as follows:

`spline = InterpolatedUnivariateSpline(x_values, y_values, k=3)`

where `x_values` is a list or numpy array containing the independent variable data, `y_values` is a list or numpy array containing the corresponding dependent variable data, `k=3` indicates a cubic spline, and `spline` is the spline object returned by the `InterpolatedUnivariateSpline` method.  

Recall that the independent variable data are the bondlengths $R$ stored in the numpy array `rs`, and the dependent variable data are the ground- and excited-state potential energy surface data stored in the `Es` array.  The transition dipole moment data is stored in the array `mu_z`. 

*Note on units* The bondlength data is stored in Angstroms, whereas the energies and transition dipole moments are all stored in atomic units.

**Question 3**  Fit a spline called `Eg_spline` to the ground-state potential energy surface data, a spline called `Ee_spline` to the excited-state potential energy surface data, and a spline called `mu_spline` to the transition dipole moment data.


In [ ]:
# fit all surfaces to a spline
# ==> Code to fit splines goes here! <== #
Eg_spline = InterpolatedUnivariateSpline(rs, Eg_array, k=3)
Ee_spline = InterpolatedUnivariateSpline(rs, Ee_array, k=3)
mu_spline = InterpolatedUnivariateSpline(rs, tdm_array, k=3)

Once the splines have been defined, they can be evaluated at a particular value(s) of $R$ using the following syntax `spline(R)`.  Note if you pass the spline a numpy array of $R$ values, it will return a numpy array of 
dependent variable values evaluated at each value of the $R$ array.  

In the following blocks, add code to evaluate `Eg_spline`, `Ee_spline` and `mu_spline` at `R_val = 2.5` Anstroms; store the values to variables named `Eg_val`, `Ee_val`, and `tdm_val` respectively.

In [ ]:
# ==> Code to evaluate Eg_spline, Ee_spline, and mu_spline at R_val = 2.5 Angstroms goes here! <== #
R_val = 2.5

Eg_val = Eg_spline(R_val)

Ee_val = Ee_spline(R_val)

tdm_val = np.absolute(mu_spline(R_val))

**Question 4** What is the transition energy in (expressed in atomic units and in electron volts) between the ground and first excited state at $R = 2.5$ Angstroms?  

*Answer in atomic units goes here!* 

*Answer in electron volts goes here!*

*Hint* The conversion factor between atomic units of energy and electron volts is $27.211 \frac{{\rm eV}}{{\rm atomic \: unit}}$.  Store the transition energy in a variable called `transition_energy_eV`.

In [ ]:
# conversion factor for au -> eV
au_to_eV = 27.211

# ==> Code to compute transition energy in atomic units and convert to eV goes here! <== #
transition_energy_au = Ee_val - Eg_val
transition_energy_eV = transition_energy_au * au_to_eV
print(transition_energy_au)
print(transition_energy_eV)

**Checking your results:** The next cell will check the values you obtained from your splines.  We will use an `assert` statement to compare your computed values to the expected values. The `assert` statement will pass if the comparison returns true, and will fail if the comparison returns false.  

**If one or more `assert` statements fail, you should go back and check your code in the last few blocks!**  

The comparison is performed by the numpy function `isclose()`.  For more information about the `isclose()` function, see [here](https://numpy.org/doc/stable/reference/generated/numpy.isclose.html).  

In [ ]:
assert np.isclose(Eg_val, _expected_Eg_val)
assert np.isclose(Ee_val, _expected_Ee_val)
assert np.isclose(tdm_val, _expected_tdm_val)
assert np.isclose(transition_energy_eV, _expected_deltaE_eV)



##### Step 2: Building the Rabi Hamiltonian

Next we will define a function that will build and return the Rabi Hamiltonian (Eq. (4)).  We will pass 
5 arguments to this function:

`A_value` : Electric field strength $A$ as defined in Eq. (3)

`omega_value` : Photon frequency $\omega$ that appears in the diagonals of Eq. (4)

`r_value` : An $R$ value at which the Rabi Hamiltoninan will be evaluated

`g_spline` : A spline fit to the ground state potential energy surface as a function of $R$

`e_spline` : A spline fit to the excited state potential energy surface as a function of $R$

`tdm_spline` : A spline fit to the transition dipole moment surface as a function of $R$

A template for this function follows:

```
def Rabi_Hamiltonian(lambda_value, omega_value, r_value, g_spline, e_spline, tdm_spline):
    """Function to compute the Rabi Hamiltonian

    Arguments
    ----------
    A_value : float
        electric field strength
        
    omega_value : float
        photon energy
        
    r_value : float
        value of the bondlength
        
    g_spline : scipy spline object
        spline that is fit to the ground-state potential energy surface
        
    e_spline : scipy spline object
        spline that is fit to the excited-state potential energy surface
        
    tdm_spline : scipy spline object
        spline that is fit to the transition dipole moment surface
        
    Returns
    -------
    H : numpy array
        3x3 Rabi Hamiltonian matrix
    """

    # create 3x3 numpy array to store the Rabi Hamiltonian
    H = np.zeros((3,3)) 
    
    # ==> Code to compute elements of the Rabi Hamiltonian <== #
    
    # return the Hamiltonian
    return H

```

In [ ]:
def Rabi_Hamiltonian(A_value, omega_value, r_value, g_spline, e_spline, tdm_spline):
    """Function to compute the Rabi Hamiltonian

    Arguments
    ----------
    A_value : float
        electric field strength
        
    omega_value : float
        photon energy
        
    r_value : float
        value of the bondlength
        
    g_spline : scipy spline object
        spline that is fit to the ground-state potential energy surface
        
    e_spline : scipy spline object
        spline that is fit to the excited-state potential energy surface
        
    tdm_spline : scipy spline object
        spline that is fit to the transition dipole moment surface
        
    Returns
    -------
    H : numpy array
        3x3 Rabi Hamiltonian matrix
    """
    
    # initialize 3x3 Hamiltonian matrix
    H = np.zeros((3,3))
    
    # diagonal entries
    H[0,0] = g_spline(r_value)
    H[1,1] = g_spline(r_value) + omega_value
    H[2,2] = e_spline(r_value)
    
    # off-diagonal entries
    H[1,2] = A_value * tdm_spline(r_value)
    H[2,1] = A_value * tdm_spline(r_value)
    
    # return the matrix
    return H


**Question 5** Evaluate the Rabi Hamiltonian with $A = 0.003$ atomic units, $R = 2.5$ Angstroms, 
and $\omega$ defined such that it matches the transition energy at this bondlength.  Store the resulting
matrix to a variable named `H_Rabi`.

*Hint:* Your $\omega$ value should be in atomic units.

In [ ]:
A_value = 0.003

H_Rabi = Rabi_Hamiltonian(A_value, transition_energy_au, R_val, Eg_spline, Ee_spline, mu_spline)


**Checking your results:** The next cell will check the Hamiltonian computed in the last block.

**If this `assert` statements fail, you should go back and check the code in your Rabi_Hamiltonian function and the values of the arguments you passed to this function!**  

In [ ]:
assert np.allclose(H_Rabi, _expected_H_Rabi)

**Step 3: Compute and plot polariton potential energy surfaces**

We will compute and plot the polaritonic potential energy surfaces for $A = 0.003$ atomic units.  This will help us to vizualize the so-called Rabi splitting that occurs between 
the surfaces at a particular value of the bondlength $R_{deg}$, where we define $R_{deg}$ as the bondlength 
at which the following is satisfied for a particular value of the photon energy $\omega_0$:

$$ E_g(R_{deg}) + \hbar \omega_0 = E_e(R_{deg}) \tag{5}$$.

If we set $\omega_0$ to be equal to the transition energy at $R = 2.5$ Angstroms as before, then we have 
$R_{deg} = 2.5$ Angstroms by definition. As we worked out in an earlier cell, this value of $\omega_0$ would
correspond to approximately $0.1487$ atomic units or $4.048$ eV.  We will use this value for $\omega$ for the rest of this notebook.

We already have a function that will build the Rabi Hamiltonian as a function of the fundamental coupling strength, the photon energy, and the bondlength value.  The next step is to build a function that will build and diagonalize
this matrix for a range of bondlengths to return the so-called lower- and upper-poloriton potential energy surfaces ($E_{LP}(R)$ and $E_{UP}(R)$).  We will use the `eigh` function of numpy's linear algebra package (given the alias `LA` in our import statement at the top of the notebook) to diagonalize the Hamiltonian and store the eigenvalues.  The syntax follows:

`vals, vecs = LA.eigh(matrix)`

where `vals` are the eigenvalues, `vecs` are the eigenvectors, and `matrix` is the Hermitian matrix.

The following function `polariton_surfaces` will compute and return $E_{LP}(R)$ and $E_{UP}(R)$ in this way.

In [ ]:
def polariton_surfaces(A_value, omega_value, r_values, g_spline, e_spline, tdm_spline):
    """Function to compute the lower- and upper-polariton potential energy surfaces

    Arguments
    ----------
    A_value : float
        electric field strength
        
    omega_value : float
        photon energy
        
    r_value : float
        value of the bondlength
        
    g_spline : scipy spline object
        spline that is fit to the ground-state potential energy surface
        
    e_splien : scipy spline object
        spline that is fit to the excited-state potential energy surface
        
    tdm_spline : scipy spline object
        spline that is fit to the transition dipole moment surface
        
    Returns
    -------
    E_LP_of_R : numpy array
        lower-polariton potential energy surface defined at each value of r_values
        
    E_UP_of_R : numpy array
        upper-polariton potential energy surface defined at each value of r_values
    """
    # initialize lp and up surfaces
    lp_surface = np.zeros_like(r_values)
    up_surface = np.zeros_like(r_values)
    
    # loop through r values, build Rabi Hamiltonian, diagonalize, and store!
    
    for i in range(0, len(r_values)):
        
        # if there is no coupling, then we don't need to diagonalize anything
        if A_value == 0:
            lp_surface[i] = g_spline(r_values[i]) + omega_value
            up_surface[i] = e_spline(r_values[i])
        
        # otherwise build Hamiltonian and diagonalize
        else:
            # Build the Rabi Hamiltonian
            H = Rabi_Hamiltonian(A_value, omega_value, r_values[i], g_spline, e_spline, tdm_spline)
            
            # diagonalize
            vals, vecs = LA.eigh(H)
            
            # store lp and up values
            lp_surface[i] = vals[1]
            up_surface[i] = vals[2]
    
    # return the surfaces
    return lp_surface, up_surface


In [ ]:
# run this cell to compute the polariton PES
lp_surface, up_surface = polariton_surfaces(A_value, transition_energy_au, rs, Eg_spline, Ee_spline, mu_spline)

In [ ]:
plt.plot(rs, Eg_spline(rs)+transition_energy_au, "r--", label="Eg + $\hbar \omega$")
plt.plot(rs, Ee_spline(rs), "b--", label="Ee")
plt.plot(rs, lp_surface, "red", label="lower polariton")
plt.plot(rs, up_surface, "blue", label="upper polariton")
plt.grid(True)
plt.legend()
plt.show()

<a id=’Derivation’></a>
### Derivation

We will consider two quantum states for the photon - no photon in the cavity, denoted by $|0\rangle$ with associated energy eigenvalue $0$
and one photon in the cavity, denoted by $|1\rangle$ with associated energy eigenvalue $\hbar \omega$.  Note that essentially shifts the photon Hamiltonian by the zero-point energy $\hbar \frac{\omega}{2}$.
We will model this system with a generalized Rabi Hamiltonian that can be written as:

$$ \hat{H} = E_g(R) |g\rangle \langle g| + E_e(R) |g\rangle \langle g| + 
\hbar \omega \hat{b} \hat{b}^{\dagger}  + \hbar {\bf A} \cdot \mu_{ge}(R) \left(\hat{b} + \hat{b}^{\dagger} \right) \left(|g\rangle \langle e| +  |e\rangle \langle g|\right). $$
The polaritonic potential energy surfaces may be obtained by building a Hamiltonian matrix in the following basis and diagonalizing as a function of the bond length $R$: $ |\phi\rangle \in \{|g,0\rangle , |g,1\rangle , |e,0\rangle, |e,1\rangle \}. $

#### Review of Dirac Bra-Ket notation
We have used Dirac's bra-ket notation for both the electronic and the photonic states.  In this notation,
$|n\rangle = \psi_n$ is known as the *ket* for state $n$, 
and $\langle m| = \psi_m^*$ is known as the *bra* for state $m$.  This applies
to both electronic and photonic states, where we are specifically
using the notation $|g\rangle$ to denote the ground electronic ket, $|e\rangle$ to denote
the first-excited electronic ket, $|0\rangle$ to denote the 0-photon ket, and $|1\rangle$ to 
denote the 1-photon ket.

An important rule in this notation is the
following:

$$ \langle n | m \rangle = \int \psi^*_n \psi_m d\tau = \delta_{nm} $$

where the Kronicker delta function $\delta_{nm}$ is equal to 1 if $n = m$ and 0 otherwise.
We can use this rule to compute different terms in the Hamiltonian matrix.


#### Review of rules for the projection operators for electronic states

The projection operators for molecular electronic states are denoted
$|n\rangle \langle m|$ where again $|n\rangle = \psi_n$ is known as the *ket* for electronic state $n$, 
and $\langle m |= \psi_m^*$ is known as the *bra* for electronic state $m$. 
In this notebook, we will restrict the electronic states to the ground state (denoted $|g\rangle$) 
and the first excited state (denoted $|e\rangle$).  
Let us examine the contribution that arises from the $E_g(R) |g\rangle \langle g|$
term in the Hamiltonian between the basis state $|g,0\rangle = |g\rangle |0\rangle$
on the bra and the ket:

\begin{align}
& \langle g|\langle0| \left( E_g(R) |g\rangle \langle g|\right)| g\rangle  |0\rangle \\
= & \langle 0|0\rangle \langle g | \left( E_g(R) |g\rangle \langle g|\right)| g\rangle \\
=& 1 \cdot \langle g | E_g(R) |g\rangle \cdot 1 \\
&= E_g(R)  \langle g | g \rangle \\
&= E_g(R)
\end{align}
where we have used the fact that the projection operator $|g\rangle \langle g|$
only acts on the electronic states, so we were able to factor out our photonic bra
and kets $\langle 0 | 0 \rangle = 1$, and then we were also able 
to bring the ground-state energy eigenvalue $E_g(R)$ outside the integral in the second-to-last step just leaving $E_g(R) \langle g | g \rangle = E_g(R)$.

##### Question 1: Combine these rules to evaluate the following:
\begin{align}
\langle g| \left(E_g(R) |g\rangle \langle g|\right)|e\rangle \\
\langle e| \left(E_g(R) |g\rangle \langle g|\right)|e\rangle\\
\langle g| \left(E_e(R) |e\rangle \langle e|\right)|e\rangle \\
\langle e| \left(E_e(R) |e\rangle \langle e|\right)|e\rangle \\
\langle e| \left({\bf A} \cdot \mu_{eg} |e\rangle \langle g|\right)|e\rangle \\
\langle e| \left({\bf A} \cdot \mu_{eg} |e\rangle \langle g|\right)|g\rangle
\end{align}

The photonic raising operators generally obey $\hat{b}^{\dagger}|n\rangle = \sqrt{n+1}|n+1\rangle$,
where $n$ denotes the number of photons that occupy the cavity.  
For the basis states in question, we have the following:
\begin{align}
\hat{b}^{\dagger}|0\rangle = |1\rangle \\
\hat{b}^{\dagger}|1\rangle = \sqrt{2}|2\rangle
\end{align}
where $|\rangle$ denotes a fermionic Fock vacuum state.

The photonic lowering operators generally obey $\hat{b}|n\rangle = \sqrt{n}|n-1\rangle$, so for the
basis states in question, we have:
\begin{align}
\hat{b}|1\rangle = |0\rangle \\
\hat{b}|0\rangle = 0.
\end{align}

##### Question 2: Combine the molecular and photonic creation and annihilation operator rules to derive elements of the Hamiltonian matrix

This matrix as a function of the bond-length $R$ is as follows:
\begin{equation}
{\bf H}(R)
  \mbox{=} 
  \begin{array}{c|cccc}
       & |g,0\rangle & |g,1\rangle & |e,0\rangle \\
    \hline
    \langle g,0| & E_g(R)   &     0   & 0  \\
    \langle g,1| & 0        &   E_g(R) +  \hbar \omega  & \hbar {\bf A} \cdot \mu_{eg} (R) \\
    \langle e,0| & 0        &    \hbar {\bf A} \cdot \mu_{eg}  & E_e(R)+\hbar \omega\\
  \end{array}
\end{equation}